In [1]:
import tkinter as tk
import os
import json
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

In [2]:
path = r'C:\Users\user\Desktop\data' 
os.chdir(path)


#Function for extracting name of each folder from the parent folder
def extract_data(path):
    folders = []
    for i,j,k in os.walk(path,topdown = False):
        for name in j:
            folders.append(name)
    


    #Iterate through all the directories and extract the data and column names for each measurement
    data = [[] for i  in range(len(folders))]
    columns_temp = []
    
    for  i in range(len(folders)):
        os.chdir(path + '\\' + str(folders[i]))
        with open('data.dat', 'r') as files:
            for file in files:
                data[i].append(file.split())
            
        with open('data.json', 'r') as column:
            listname = json.load(column)
            columns_temp.append(listname)
     
    #Generate the columns fro each data set from the dictionaries
    columns = [[] for  i in range(len(columns_temp))]
    for i in range(len(columns_temp)):
        for j in columns_temp[i]:
            columns[i].append(str(j.values())[str(j.values()).index('[')+1:str(j.values()).index(']')])
            
            

    return folders, columns, data


folders, columns, data = extract_data(path)       

In [3]:
#Tkinter class for the gui: L variables are labels, E variables are entries, D variables are Optionmenu and B variables
#are buttons
class cryo_reader:
    def __init__(self,window):
        self.window = window
        window.title('cryo_reader')
        self.x = []
        self.y = []
        
        self.L1 = tk.Label(window, text = 'Select Folder') 
        self.L1.grid(row =0, column = 0)
        
        self.v1 = tk.StringVar()#Stringvar that will be used for the name of the folder selected in Optionmenu 1 (D1)
        self.index_folder = tk.IntVar()#Intvar that will be used for the index of the folder
        self.new_columns = columns[self.index_folder.get()]#new_columns list that will change for each folder
        
        self.D1 = tk.OptionMenu(window,self.v1, *folders)
        self.D1.grid(row = 0, column = 1)
        
        self.L2 = tk.Label(window, text = 'Select x axis')
        self.L2.grid(row = 1, column = 0)
        
        self.v2 = tk.StringVar() 
        self.v2.set("X axis")
        
        self.D2 = tk.OptionMenu(window,self.v2, *self.new_columns)
        self.D2.grid(row = 1, column = 1)
        
        self.L3 = tk.Label(window, text = 'Select y axis')
        self.L3.grid(row = 2, column = 0)
        
        self.v3 = tk.StringVar()
        self.v3.set("y axis")
        
        self.D3 = tk.OptionMenu(window,self.v3, *self.new_columns)
        self.D3.grid(row = 2, column = 1)
        
        self.B1 = tk.Button(window, text = 'Select folder', command = self.go_to_folder)
        self.B1.grid(row = 0, column = 2) #Button for selecting the folder chosen in the folders dropdown menu
        
        self.B2 = tk.Button(window, text = 'Select x', command = self.select_val_x)
        self.B2.grid(row = 1, column = 2) #Button for selecting the column for the x_axis in the columns dropdown menu

        self.B3 = tk.Button(window, text = 'Select y', command = self.select_val_y)
        self.B3.grid(row = 2, column = 2) #Button for selecting the column for the y_axis in the columns dropdown menu
        
        self.B4 = tk.Button(window, text = "Plot", command = self.plot_figure)
        self.B4.grid(row = 0, column = 3) #Button for ploting the x and y values
        
        self.B4 = tk.Button(window, text = "One way", command = self.one_or_both)
        self.B4.grid(row = 1, column = 3) #Button for selecting only the first half of the data samples. (In transport 
        #measurements sweeps are performed twice)
        
        self.B5 = tk.Button(window, text = "Set axis to 0", command = self.set_axis)
        self.B5.grid(row = 2, column = 3) # Button for setting the lower value for x at 0. In certain measurements, delay of 
        # the signal can cause an ofset
        
        self.L4 = tk.Label(window,text = "Window_length") #Window length for the filter. It must be an odd number.
        self.L4.grid(row = 0, column = 4)
        
        self.L5 = tk.Label(window,text = "Polyorder") #Polynomial order for the filter. It must be lower than the window_length.
        self.L5.grid(row = 1, column = 4)
        
        self.E1 = tk.Entry(window)
        self.E1.grid(row = 0, column = 5)
        
        self.E2 = tk.Entry(window)
        self.E2.grid(row = 1, column = 5)
        
        self.B6 = tk.Button(window, text = "Filter", command = self.smooth)
        self.B6.grid(row = 2, column = 5) #Button for decreasing the noise of the signal with the Savitzky-Goley filter. 
        
    def go_to_folder(self):
        
        #Function for selecting a folder from the D1 dropdown menu and selecting the columns for that folder in the 
        #D2 and D3 dropdown menues
        
        self.index_folder = folders.index(str(self.v1.get()))
        os.chdir(path + '\\' + str(folders[self.index_folder]))
        
        self.new_columns = columns[self.index_folder]
            
        self.v2.set("X axis")
        self.D2.destroy()
            
        self.D2 = tk.OptionMenu(window,self.v2, *self.new_columns)
        self.D2.grid(row = 1, column = 1)
        
        self.v3.set("y axis")
        self.D3.destroy()
        
        self.D3 = tk.OptionMenu(window,self.v3, *self.new_columns)
        self.D3.grid(row = 2, column = 1)
        
       
    def select_val_x(self):
        #Function for extracting the x_axis from data 
        self.index_x = self.new_columns.index(self.v2.get())

        for i in data[self.index_folder]:
            self.x.append(float(i[self.index_x]))
        
    def select_val_y(self):
        #Function for extracting the y_axis from data 
        self.index_y = self.new_columns.index(self.v3.get())

        for i in data[self.index_folder]:
            self.y.append(float(i[self.index_y]))
            
    
    def plot_figure(self):
        figure1 = plt.Figure(figsize=(10,6))
        ax1 = figure1.add_subplot()
        line1 = FigureCanvasTkAgg(figure1, window)
        line1.get_tk_widget().grid(row = 3, column = 0,columnspan = 6)
        ax1.set_title(self.v1.get())
        ax1.set_xlabel(self.v2.get())
        ax1.set_ylabel(self.v3.get())
        ax1.plot(self.x,self.y)
        
        
        self.x = []
        self.y = []
      
    def one_or_both(self):
        #Function for selecting half the sample
        self.x = self.x[len(self.x)//2:]
        self.y = self.y[len(self.y)//2:]
        
    def set_axis(self):
        #Function for setting x to 0 where y(x) is the minimum
        self.x = [i - self.x[self.y.index(min(self.y))] for i in self.x]
        
    def smooth(self):
        #Function for applying the savitzky Goley filter for noise reduction
        self.window_length = int(self.E1.get())
        self.polyorder = int(self.E2.get())
        
        self.y = savgol_filter(self.y,self.window_length,self.polyorder)
    

In [ ]:
window = tk.Tk()

app = cryo_reader(window)
window.mainloop()